# ICR - Identifying Age Related Conditions (Kaggle)

## 1. Imports

In [14]:
pip install tabpfn -q

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from imblearn.over_sampling import RandomOverSampler
from tqdm.notebook import tqdm
from sklearn.model_selection import KFold as KF
import xgboost
from tabpfn import TabPFNClassifier

## 2. Training Data

In [3]:
! wget https://raw.githubusercontent.com/mathschelsea/data_science/feature/kaggle_icr/data/external/ICR/train.csv
train_raw = pd.read_csv('train.csv')

pd.set_option('display.max_columns', None)
print(f'Train shape: {train_raw.shape}')
train_raw.head()

--2024-02-25 19:16:55--  https://raw.githubusercontent.com/mathschelsea/data_science/feature/kaggle_icr/data/external/ICR/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 336555 (329K) [text/plain]
Saving to: ‘train.csv.1’

train.csv.1         100%[===================>] 328.67K  --.-KB/s    in 0.03s   

2024-02-25 19:16:55 (9.73 MB/s) - ‘train.csv.1’ saved [336555/336555]

Train shape: (617, 58)


,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,BN,BP,BQ,BR,BZ,CB,CC,CD,CF,CH,CL,CR,CS,CU,CW,DA,DE,DF,DH,DI,DL,DN,DU,DV,DY,EB,EE,EG,EH,EJ,EL,EP,EU,FC,FD,FE,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,4126.58731,22.5984,175.638726,152.707705,823.928241,257.432377,47.223358,0.563481,23.387600,4.851915,0.023482,1.050225,0.069225,13.784111,1.302012,36.205956,69.08340,295.570575,0.23868,0.284232,89.245560,84.31664,29.657104,5.310690,1.74307,23.187704,7.294176,1.987283,1433.166750,0.949104,B,30.879420,78.526968,3.828384,13.394640,10.265073,9028.291921,3.583450,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,5496.92824,19.4205,155.868030,14.754720,51.216883,257.432377,30.284345,0.484710,50.628208,6.085041,0.031442,1.113875,1.117800,28.310953,1.357182,37.476568,70.79836,178.553100,0.23868,0.363489,110.581815,75.74548,37.532000,0.005518,1.74307,17.222328,4.926396,0.858603,1111.287150,0.003042,A,109.125159,95.415086,52.260480,17.175984,0.296850,6785.003474,10.358927,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,5135.78024,26.4825,128.988531,219.320160,482.141594,257.432377,32.563713,0.495852,85.955376,5.376488,0.036218,1.050225,0.700350,39.364743,1.009611,21.459644,70.81970,321.426625,0.23868,0.210441,120.056438,65.46984,28.053464,1.289739,1.74307,36.861352,7.813674,8.146651,1494.076488,0.377208,B,109.125159,78.526968,5.390628,224.207424,8.745201,8338.906181,11.626917,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,4169.67738,23.6577,237.282264,11.050410,661.518640,257.432377,15.201914,0.717882,88.159360,2.347652,0.029054,1.400300,0.636075,41.116960,0.722727,21.530392,47.27586,196.607985,0.23868,0.292431,139.824570,71.57120,24.354856,2.655345,1.74307,52.003884,7.386060,3.813326,15691.552180,0.614484,B,31.674357,78.526968,31.323372,59.301984,7.884336,10965.766040,14.852022,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,5728.73412,24.0108,324.546318,149.717165,6074.859475,257.432377,82.213495,0.536467,72.644264,30.537722,0.025472,1.050225,0.693150,31.724726,0.827550,34.415360,74.06532,200.178160,0.23868,0.207708,97.920120,52.83888,26.019912,1.144902,1.74307,9.064856,7.350720,3.490846,1403.656300,0.164268,B,109.125159,91.994825,51.141336,29.102640,4.274640,16198.049590,13.666727,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1


In [4]:
# Label counts
train_raw.Class.value_counts()

0    509
1    108
Name: Class, dtype: int64

In [5]:
# Checking out the only categorical column, EJ
train_raw.EJ.value_counts()

B    395
A    222
Name: EJ, dtype: int64

In [6]:
# Transforming column EJ from categorical to numerical
cat1 = train_raw.EJ.unique()[0]
train_raw.EJ = train_raw.EJ.eq(cat1).astype(int)
train_raw.EJ.value_counts()

1    395
0    222
Name: EJ, dtype: int64

In [7]:
# Importing the additional dataset
! wget https://raw.githubusercontent.com/mathschelsea/data_science/feature/kaggle_icr/data/external/ICR/greeks.csv
greeks = pd.read_csv('greeks.csv')

print(f'Greeks shape: {greeks.shape}')
greeks.head()

--2024-02-25 19:16:55--  https://raw.githubusercontent.com/mathschelsea/data_science/feature/kaggle_icr/data/external/ICR/greeks.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18853 (18K) [text/plain]
Saving to: ‘greeks.csv.1’

greeks.csv.1        100%[===================>]  18.41K  --.-KB/s    in 0.001s  

2024-02-25 19:16:56 (14.8 MB/s) - ‘greeks.csv.1’ saved [18853/18853]

Greeks shape: (617, 6)


,Id,Alpha,Beta,Gamma,Delta,Epsilon
0,000ff2bfdfe9,B,C,G,D,3/19/2019
1,007255e47698,A,C,M,B,Unknown
2,013f2bd269f5,A,C,M,B,Unknown
3,043ac50845d5,A,C,M,B,Unknown
4,044fb8a146ec,D,B,F,B,3/25/2020


In [8]:
# Using the Epsilon column
dates = greeks.Epsilon.copy()

# Adds a time element of 00:00:00 to Epsilon and converts it to a proleptic Gregorian ordinal of the datetime
dates[greeks.Epsilon != 'Unknown'] = greeks.Epsilon[greeks.Epsilon != 'Unknown'].map(lambda x: datetime.strptime(x,'%m/%d/%Y').toordinal())
dates[greeks.Epsilon == 'Unknown'] = np.nan
dates = dates.astype(float)

# Putting together the training dataset
train = pd.concat((train_raw, dates), axis=1)

## 3. Over-Sampling

In [9]:
# Oversample the positive responses based on the Alpha column
ros = RandomOverSampler(random_state=37)
train_ros, y_ros = ros.fit_resample(train, greeks.Alpha)

print('Original dataset shape')
print(greeks.Alpha.value_counts())
print('Resample dataset shape')
print(y_ros.value_counts())

Original dataset shape
A    509
B     61
G     29
D     18
Name: Alpha, dtype: int64
Resample dataset shape
B    509
A    509
D    509
G    509
Name: Alpha, dtype: int64


In [10]:
train_ros_x = train_ros.drop(['Id', 'Class'], axis=1)
train_ros_y = train_ros.Class

## 4. Model Training

In [11]:
def balanced_log_loss(y_true, y_pred):
    # y_true: correct labels 0, 1
    # y_pred: predicted probabilities of class=1
    # calculate the number of observations for each class
    N_0 = np.sum(1 - y_true)
    N_1 = np.sum(y_true)
    # calculate the weights for each class to balance classes
    w_0 = 1 / N_0
    w_1 = 1 / N_1
    # calculate the predicted probabilities for each class
    p_1 = np.clip(y_pred, 1e-15, 1 - 1e-15)
    p_0 = 1 - p_1
    # calculate the summed log loss for each class
    log_loss_0 = -np.sum((1 - y_true) * np.log(p_0))
    log_loss_1 = -np.sum(y_true * np.log(p_1))
    # calculate the weighted summed logarithmic loss
    # (factgor of 2 included to give same result as LL with balanced input)
    balanced_log_loss = 2*(w_0 * log_loss_0 + w_1 * log_loss_1) / (w_0 + w_1)
    # return the average log loss
    return balanced_log_loss/(N_0+N_1)

In [ ]:
# K-Fold cross-validator - 5 folds
cv_inner = KF(n_splits = 5, shuffle=True, random_state=37)

# Models
models = [xgboost.XGBClassifier(),
          xgboost.XGBClassifier(n_estimators=100,max_depth=3,learning_rate=0.2,subsample=0.9,colsample_bytree=0.85),
          TabPFNClassifier(device='cuda', N_ensemble_configurations=24),
          TabPFNClassifier(device='cuda', N_ensemble_configurations=64)]

# Training loop
results = list()

for m in models:

  split = 0
  losses = list()

  for train_idx, test_idx in cv_inner.split(train_ros_x):

    # Split counter
    split+=1

    # Train and test splits
    x_train = train_ros_x.iloc[train_idx]
    y_train = train_ros_y.iloc[train_idx]
    x_test = train_ros_x.iloc[test_idx]
    y_test = train_ros_y.iloc[test_idx]

    # Train model
    if m == models[2] or m == models[3]:
      m.fit(x_train, y_train, overwrite_warning=True)
    else:
      m.fit(x_train, y_train)

    # Predict the class probabilities
    probs = m.predict_proba(x_test)
    # Take Class=0 probabilities
    p0 = probs[:,:1]
    # Format predictions
    y_pred = np.empty((probs.shape[0],))
    for i in range(probs.shape[0]):
      if p0[i]>=0.5:
        y_pred[i]= False
      else:
        y_pred[i]=True
    y_pred = y_pred.astype(int)

    # Calculate competition evaluation metric
    loss = balanced_log_loss(y_test, y_pred)

    # Collect loss outcomes
    losses.append(loss)
    #print(f'Split {split} Loss = {loss}')

  # Average loss for each model
  results.append(np.mean(losses))

## 5. Model Results

In [13]:
for n in range(len(models)):
  print(f'Model {n} Loss: {results[n]}');

Model 0 Loss: 0.4340741973457757
Model 1 Loss: 0.459386315120278
Model 2 Loss: 0.230360391549934
Model 3 Loss: 0.25529826191850835


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
